This is an example notebook on how to use NAM library for shape matching

In [6]:
# import libraries
import sys 
sys.path.append('..')
from model.neural_adjoint_map import *
from model.optimizer import *


In [7]:
#load data
import yaml
sys.path.append('./../../NRP/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/')
import logging
from os import path as osp

from datasets import build_dataset
from models import build_model

import yaml
from utils.tensor_util import to_device,to_numpy
import torch
import numpy as np

#torch.cuda.set_device(0)

#pointcloud

DATASET='smal'

with open('./../../NRP/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/options/test/'+DATASET+'.yaml', 'r') as f:
    opt = yaml.load(f, Loader=yaml.SafeLoader)
opt['is_train']=False
test_set = build_dataset(opt['datasets']['test_dataset'])

shape1=test_set[0]['first']
shape2=test_set[0]['second']

In [8]:
nam12=Neural_Adjoint_Map(3)
loss_handler=LossHandler()
opt=NAMOptimizer(nam12,loss_handler)
opt.optimize(shape1['verts'][shape1['corr']],shape2['verts'][shape2['corr']])

Iteration 1/200, Loss: 0.10583597421646118
Iteration 11/200, Loss: 0.031553544104099274
Iteration 21/200, Loss: 0.0050303214229643345
Iteration 31/200, Loss: 0.0016742735169827938
Iteration 41/200, Loss: 0.0003125985385850072
Iteration 51/200, Loss: 0.00015387138410005718
Iteration 61/200, Loss: 7.987720164237544e-05
Iteration 71/200, Loss: 2.6970838007400744e-05
Iteration 81/200, Loss: 2.0322733689681627e-05
Iteration 91/200, Loss: 1.1503139830892906e-05
Iteration 101/200, Loss: 8.252738552982919e-06
Early stopping at iteration 105


Neural_Adjoint_Map(
  (fmap_branch): Linear(in_features=3, out_features=3, bias=False)
  (nonlinear_branch): Sequential(
    (0): Linear(in_features=3, out_features=128, bias=False)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=128, out_features=128, bias=False)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=128, out_features=128, bias=False)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=128, out_features=128, bias=False)
    (7): LeakyReLU(negative_slope=0.01)
    (8): Linear(in_features=128, out_features=3, bias=False)
  )
)

In [9]:
from methods.linear_ref import *

In [10]:
import torch_cluster

OSError: [WinError 127] Impossibile trovare la procedura specificata